In [1]:
import os
import tempfile

import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Normalize, Compose

import ray.train.torch

def train_func():
    # Model, Loss, Optimizer
    model = resnet18(num_classes=10)
    model.conv1 = torch.nn.Conv2d(
        1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    )
    # [1] Prepare model.
    model = ray.train.torch.prepare_model(model)
    # model.to("cuda")  # This is done by `prepare_model`
    criterion = CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.001)

    # Data
    transform = Compose([ToTensor(), Normalize((0.5,), (0.5,))])
    data_dir = os.path.join(tempfile.gettempdir(), "data")
    train_data = FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
    train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
    # [2] Prepare dataloader.
    train_loader = ray.train.torch.prepare_data_loader(train_loader)

    # Training
    for epoch in range(10):
        if ray.train.get_context().get_world_size() > 1:
            train_loader.sampler.set_epoch(epoch)

        for images, labels in train_loader:
            # This is done by `prepare_data_loader`!
            # images, labels = images.to("cuda"), labels.to("cuda")
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # [3] Report metrics and checkpoint.
        metrics = {"loss": loss.item(), "epoch": epoch}
        with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
            torch.save(
                model.module.state_dict(),
                os.path.join(temp_checkpoint_dir, "model.pt")
            )
            ray.train.report(
                metrics,
                checkpoint=ray.train.Checkpoint.from_directory(temp_checkpoint_dir),
            )
        if ray.train.get_context().get_world_rank() == 0:
            print(metrics)

# [4] Configure scaling and resource requirements.
scaling_config = ray.train.ScalingConfig(num_workers=2, use_gpu=False)

# [5] Launch distributed training job.
trainer = ray.train.torch.TorchTrainer(
    train_func,
    scaling_config=scaling_config,
    # [5a] If running in a multi-node cluster, this is where you
    # should configure the run's persistent storage that is accessible
    # across all worker nodes.
    # run_config=ray.train.RunConfig(storage_path="s3://..."),
)
result = trainer.fit()

# [6] Load the trained model.
with result.checkpoint.as_directory() as checkpoint_dir:
    model_state_dict = torch.load(os.path.join(checkpoint_dir, "model.pt"))
    model = resnet18(num_classes=10)
    model.conv1 = torch.nn.Conv2d(
        1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    )
    model.load_state_dict(model_state_dict)

(RayTrainWorker pid=75027) Setting up process group for: env:// [rank=0, world_size=2]
(TorchTrainer pid=75003) Started distributed worker processes: 
(TorchTrainer pid=75003) - (ip=127.0.0.1, pid=75027) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=75003) - (ip=127.0.0.1, pid=75028) world_rank=1, local_rank=1, node_rank=0
(RayTrainWorker pid=75028) [W ProcessGroupGloo.cpp:751] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
(RayTrainWorker pid=75027) Moving model to device: cpu
(RayTrainWorker pid=75027) Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=75028) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=75028) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /var/folders/0n/0gc794z54nv7z0mdwxbh998h0000gp/T/data/FashionMNIST/raw/train-images-idx3-ubyte.gz


 44%|████▍     | 11665408/26421880 [00:03<00:03, 4759212.57it/s]
(RayTrainWorker pid=75027) [W ProcessGroupGloo.cpp:751] Warning: Unable to resolve hostname to a (local) address. Using the loopback address as fallback. Manually set the network interface to bind to with GLOO_SOCKET_IFNAME. (function operator())
100%|██████████| 26421880/26421880 [00:05<00:00, 5237262.29it/s]


(RayTrainWorker pid=75028) Extracting /var/folders/0n/0gc794z54nv7z0mdwxbh998h0000gp/T/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /var/folders/0n/0gc794z54nv7z0mdwxbh998h0000gp/T/data/FashionMNIST/raw
(RayTrainWorker pid=75027) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=75027) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /var/folders/0n/0gc794z54nv7z0mdwxbh998h0000gp/T/data/FashionMNIST/raw/train-images-idx3-ubyte.gz


 83%|████████▎ | 21987328/26421880 [00:05<00:00, 7749730.09it/s] [repeated 60x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


...

(RayTrainWorker pid=75028) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/Xingsheng_Qian/ray_results/TorchTrainer_2024-03-29_11-23-32/TorchTrainer_7458a_00000_0_2024-03-29_11-23-34/checkpoint_000008) [repeated 2x across cluster]


(RayTrainWorker pid=75027) {'loss': 0.13146929442882538, 'epoch': 8}


(RayTrainWorker pid=75028) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/Xingsheng_Qian/ray_results/TorchTrainer_2024-03-29_11-23-32/TorchTrainer_7458a_00000_0_2024-03-29_11-23-34/checkpoint_000009) [repeated 2x across cluster]


(RayTrainWorker pid=75027) {'loss': 0.04939134791493416, 'epoch': 9}


2024-03-29 12:03:32,897	INFO tune.py:1042 -- Total run time: 2397.93 seconds (2397.91 seconds for the tuning loop).


In [1]:
scaling_config = ray.train.ScalingConfig(num_workers=2, use_gpu=True)

NameError: name 'ray' is not defined